In [ ]:
import unicodedata
import pandas as pd
import xml.etree.ElementTree as et
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
from matplotlib.colors import BoundaryNorm
from mapclassify import FisherJenks

def normalizar_sin_tildes(texto):
    """
    Convierte un texto a mayúsculas, eliminando tildes,
    pero manteniendo caracteres especiales como ñ, Ñ, ü, Ü, ç, etc.
    """
    # Normaliza en forma NFD (separa letras y tildes)
    texto_normalizado = unicodedata.normalize('NFD', texto)
    
    # Filtra las marcas diacríticas (tildes), pero conserva la virgulilla (~) de la ñ y la diéresis (¨) de la ü
    texto_sin_tildes = ''.join(
        c for c in texto_normalizado 
        if unicodedata.category(c) != 'Mn' or c in ['\u0303', '\u0308']  # ~ y ¨
    )
    
    # Vuelve a normalizar en NFC y convierte a mayúsculas
    return unicodedata.normalize('NFC', texto_sin_tildes).upper()

tipo_colegio = "Municipal"

data = pd.read_csv("grouped_data (1).csv")
data = data[data["COD_DEPE2_desc"] == tipo_colegio]

In [56]:
print(data["desertion_rate"].min(),data["desertion_rate"].max())

0.125 1.0


In [57]:
# 2. Calcular límites de cuartiles (0%, 25%, 50%, 75%, 100%)
quantile_bounds = np.percentile(data['desertion_rate'], [0, 20, 40, 60, 80, 100])

# 4. Definir colormap y normalizador uniforme y por cuantiles
colormap = plt.cm.OrRd

norm = plt.Normalize(vmin=0.125, vmax=0.9)
data['color_hex'] = data['desertion_rate'].apply(lambda x: mcolors.to_hex(colormap(norm(x))))

norm = BoundaryNorm(quantile_bounds, ncolors=colormap.N)
data['color_hex_quantiles'] = data['desertion_rate'].apply(
    lambda x: mcolors.to_hex(colormap(norm(x)))
)

# Clasificación Fisher-Jenks
fisher_jenks = FisherJenks(data['desertion_rate'], k=7)
norm = BoundaryNorm(fisher_jenks.bins, ncolors=colormap.N)

# Asignar color
data['color_hex_fisher'] = data['desertion_rate'].apply(
    lambda x: mcolors.to_hex(colormap(norm(x)))
)

tree = et.parse("Comunas_de_Santiago.svg")
root = tree.getroot()

modelo = ["color_hex", "color_hex_quantiles", "color_hex_fisher"]
modelo_seleccionado = 1

ns = {'svg': 'http://www.w3.org/2000/svg'}

for elem in root.findall('.//svg:path', ns):

    style = elem.get('style')
    id = elem.get("id")
    if id is None:
        continue

    id_df = normalizar_sin_tildes(id)

    try:
        color = data.loc[data["comuna"] == id_df, modelo[modelo_seleccionado - 1]].values[0]
    except Exception:
        color = "url(#hatch)"

    elem.set("style",f"fill:{color};fill-opacity:1;fill-rule:evenodd;stroke:black;stroke-width:0.2;stroke-linecap:butt;stroke-linejoin:miter;stroke-miterlimit:4;stroke-dasharray:none;stroke-opacity:1")

tree.write(f"mapa_heatmap_{tipo_colegio}_{modelo[modelo_seleccionado - 1]}.svg")

c:\Users\bet0x\visualizacion\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3670: UserWarning: Numba not installed. Using slow pure python version.
  exec(code_obj, self.user_global_ns, self.user_ns)
